In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import re
from unidecode import unidecode


tree = ET.parse('intertass-ES-train-tagged.xml')
root = tree.getroot()

tweets_data = []

for tweet in root.findall('tweet'):
    tweetid = tweet.find('tweetid').text
    user = tweet.find('user').text
    content = tweet.find('content').text
    date = tweet.find('date').text
    lang = tweet.find('lang').text
    sentiment_value = tweet.find('sentiment/polarity/value').text
    
    tweets_data.append({'Tweet ID': tweetid, 'Usuario': user, 'Contenido': content,
                        'Fecha': date, 'Lenguaje': lang, 'Sentimiento': sentiment_value})

df = pd.DataFrame(tweets_data)

In [2]:
df.shape

(1008, 6)

In [3]:
df.head()

,Tweet ID,Usuario,Contenido,Fecha,Lenguaje,Sentimiento
0,768213876278165504,OnceBukowski,-Me caes muy bien \n-Tienes que jugar más part...,2016-08-23 22:30:35,es,NONE
1,768213567418036224,anahorxn,@myendlesshazza a. que puto mal escribo\n\nb. ...,2016-08-23 22:29:21,es,N
2,768212591105703936,martitarey13,@estherct209 jajajaja la tuya y la d mucha gen...,2016-08-23 22:25:29,es,N
3,768221670255493120,endlessmilerr,Quiero mogollón a @AlbaBenito99 pero sobretodo...,2016-08-23 23:01:33,es,P
4,768221021300264964,JunoWTFL,Vale he visto la tia bebiendose su regla y me ...,2016-08-23 22:58:58,es,N


In [4]:
df.Sentimiento.value_counts()

Sentimiento
N       418
P       318
NONE    139
NEU     133
Name: count, dtype: int64

In [5]:
df_sentiment = df[df['Sentimiento']!='NONE'].reset_index(drop=True)

In [6]:
df_sentiment.Sentimiento.value_counts()

Sentimiento
N      418
P      318
NEU    133
Name: count, dtype: int64

In [7]:
df_sentiment.shape

(869, 6)

In [7]:
df_sentiment.Contenido.iloc[0]

'@myendlesshazza a. que puto mal escribo\n\nb. me sigo surrando help \n\n3. ha quedado raro el "cómetelo" ahí JAJAJAJA'

# Procesamiento

In [9]:
def cargar_stopwords(archivo_stopwords):
    with open(archivo_stopwords, 'r', encoding='utf-8') as f:
        stopwords = f.read().splitlines()
    return stopwords

def limpiar_texto(texto):
    # eliminar intros y otros caracteres no deseados
    texto_limpio = re.sub(r'@[\w.-]+', '', texto)
    texto_limpio = re.sub(r'\n', ' ', texto_limpio)  # eliminar intros
    texto_limpio = re.sub(r'[^\w\s]', '', texto_limpio)  # eliminar caracteres especiales excepto puntos
    # cambiar tildes por la misma letra sin tilde
    # texto_limpio = unidecode(texto_limpio)
    # convertir texto a minúsculas
    texto_limpio = texto_limpio.lower()
    # tokenizar el texto en palabras
    palabras = texto_limpio.split()
    # eliminar stopwords
    palabras_filtradas = [palabra for palabra in palabras if palabra not in stopwords_espanol]
    # reconstruir el texto sin las stopwords
    texto_sin_stopwords = ' '.join(palabras_filtradas)
    return texto_sin_stopwords

# cargar stopwords en español
stopwords_espanol = cargar_stopwords('spanish_stop_words.txt')


In [10]:
df_sentiment['texto_limpio'] = df_sentiment['Contenido'].apply(limpiar_texto)

In [11]:
mapping = {'N': "negative", 'NEU': "neutral", 'P': "positive"}
df_sentiment['Sentimiento'] = df_sentiment['Sentimiento'].replace(mapping)

In [12]:
df_sentiment.head()

,Tweet ID,Usuario,Contenido,Fecha,Lenguaje,Sentimiento,texto_limpio
0,768213567418036224,anahorxn,@myendlesshazza a. que puto mal escribo\n\nb. ...,2016-08-23 22:29:21,es,negative,puto mal escribo b sigo surrando help 3 quedad...
1,768212591105703936,martitarey13,@estherct209 jajajaja la tuya y la d mucha gen...,2016-08-23 22:25:29,es,negative,jajajaja d mucha gente seguro puedo melena muero
2,768221670255493120,endlessmilerr,Quiero mogollón a @AlbaBenito99 pero sobretodo...,2016-08-23 23:01:33,es,positive,quiero mogollón sobretodo rápido contesta wasaps
3,768221021300264964,JunoWTFL,Vale he visto la tia bebiendose su regla y me ...,2016-08-23 22:58:58,es,negative,vale visto tia bebiendose regla hs dado muchs ...
4,768220253730009091,Alis_8496,@Yulian_Poe @guillermoterry1 Ah. mucho más por...,2016-08-23 22:55:55,es,positive,ah supuesto solo incluyo entendido mal


In [14]:
df_sentiment.to_csv('tass_clean.csv', index=False)